# BM25 and BERT Retrieval

### Step 1: Import everything and load variables

In [1]:
#pip install python-terrier

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib64/jvm/java"
os.environ["PYTERRIER_VERSION"] = "snapshot"
os.environ["PYTERRIER_HELPER_VERSION"] = "snapshot"
import pyterrier as pt
pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
#import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded, get_input_directory_and_output_directory, persist_and_normalize_run
import json
from tqdm import tqdm

ensure_pyterrier_is_loaded()
input_directory, output_directory = get_input_directory_and_output_directory('./iranthology-dataset-tira')


I will use a small hardcoded example located in ./iranthology-dataset-tira.
The output directory is /tmp/


### Step 2: Load the Data

In [4]:
print('Step 2: Load the data.')

queries = pt.io.read_topics('milestone-1-topics.xml', format='trecxml')

documents = [json.loads(i) for i in open(input_directory + '/documents.jsonl', 'r')]
documents = [{'docno': i['docno'], 'text': i['text'], 'title': i['original_document']['title'], 'abstract': i['original_document']['abstract']} for i in documents]

print('We look at the first document:\n')
print(documents[0])

Step 2: Load the data.
We look at the first document:

{'docno': '2019.sigirconf_workshop-2019birndl.0', 'text': 'Proceedings of the 4th Joint Workshop on Bibliometric-enhanced Information Retrieval and Natural Language Processing for Digital Libraries (BIRNDL 2019) co-located with the 42nd International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR 2019), Paris, France, July 25, 2019 ', 'title': 'Proceedings of the 4th Joint Workshop on Bibliometric-enhanced Information Retrieval and Natural Language Processing for Digital Libraries (BIRNDL 2019) co-located with the 42nd International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR 2019), Paris, France, July 25, 2019', 'abstract': ''}


### Step 3: Create the Index

In [5]:
print('Step 3: Create the Index.')

!rm -Rf ./index
iter_indexer = pt.IterDictIndexer("./index", meta={'docno' : 100, 'title': 10240, 'abstract': 10240, 'text': 10240}, blocks=True)
index_ref = iter_indexer.index(tqdm(documents))

Step 3: Create the Index.
rm: das Entfernen von './index' ist nicht möglich: Das Verzeichnis ist nicht leer


 31%|███████████████████▊                                           | 16873/53673 [00:05<00:06, 5703.70it/s]

13:49:02.908 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


100%|███████████████████████████████████████████████████████████████| 53673/53673 [00:13<00:00, 4024.89it/s]


13:49:13.040 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


### Step 4: Create Retrieval Pipeline

Bert is a neural ranking model, which gives more attention to the semantic meaning of words instead of just word similarity. Words don't need to be syntactically similar, but rather used in the same context, to be recognized as semantically similar. 

The title and the abstract part usually include key words, so we sticked with the BM25 model there. The text might name the keyword (in this context the query) only once, but still be relevant, which is the reason we thought BERT would be effective.

In [20]:
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", verbose=True, metadata=['docno', 'text', 'title', 'abstract'])

bm25_title = pt.text.scorer(body_attr="title", wmodel="BM25")
bm25_abstract = pt.text.scorer(body_attr="abstract", wmodel="BM25")
bert_text = pt.text.scorer(body_attr="text", wmodel="BERT")

# We wanted to pay less attention to the abtract, since many documents dont have one. 

combined_score = ((2*bm25_title) + (1*bm25_abstract) + (2*bert_text))

#We decreased the top 1000 results to top 100. Only the most relevant documents are important
#a user would check only the first results anyways.

retrieval_pipeline = bm25 %100 >> combined_score

In [21]:
print('We show the first ten query document pairs after BM25 retrieval to show what fields we have added:')
run = bm25(queries[:1])
run.head(5)

We show the first ten query document pairs after BM25 retrieval to show what fields we have added:


BR(BM25): 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.43q/s]


qid  docid                               docno   
0   1  14568        2019.wsdm_conference-2019.40  \
1   1   7566  2008.sigirconf_conference-2008.116   
2   1  21754       2020.cikm_conference-2020.118   
3   1  33491   2018.wwwconf_conference-2018c.140   
4   1  14643       2019.wsdm_conference-2019.115   

                                                text   
0  Beyond News Contents: The Role of Social Conte...  \
1  Detecting synonyms in social tagging systems t...   
2  FANG: Leveraging Social Context for Fake News ...   
3  Fake News Detection in Social Networks via Cro...   
4  Fake News: Fundamental Theories, Detection Str...   

                                               title   
0  Beyond News Contents: The Role of Social Conte...  \
1  Detecting synonyms in social tagging systems t...   
2  FANG: Leveraging Social Context for Fake News ...   
3  Fake News Detection in Social Networks via Cro...   
4  Fake News: Fundamental Theories, Detection Str...   

                                            abstract  rank      score   
0                                                        0  29.858420  \
1  ABSTRACTCollaborative tagging used in online s...     1  28.648963   
2                                                        2  28.378120   
3  ABSTRACTOur work considers leveraging crowd si...     3  26.846009   
4  ABSTRACTThe explosive growth of fake news and ...     4  25.784173   

                                               query  
0   relevant documents show the bigram of fake ne...  
1   relevant documents show the bigram of fake ne...  
2   relevant documents show the bigram of fake ne...  
3   relevant documents show the bigram of fake ne...  
4   relevant documents show the bigram of fake ne...

### Step 5: Create the run

In [18]:
print('Step 5: Create Run.')

run = retrieval_pipeline(queries)

Step 5: Create Run.


BR(BM25): 100%|████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.73q/s]
Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/u/ir28jymu/.local/lib/python3.10/site-packages/pyterrier/index.py", line 720, in next
    lastdoc = self.convertFn(text, meta)
  File "/u/ir28jymu/.local/lib/python3.10/site-packages/pyterrier/index.py", line 628, in convertDoc
    for column, value in meta_column[1].iteritems():
  File "/u/ir28jymu/.local/lib/python3.10/site-packages/pandas/core/generic.py", line 5989, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'Series' object has no attribute 'iteritems'
Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/u/ir28jymu/.l

KeyError: '2019.wsdm_conference-2019.40'

In [19]:
print('We look at the first 10 results of the run (query has ben expanded):\n')
run.head(10)

We look at the first 10 results of the run (query has ben expanded):



qid  docid                               docno   
0   1  14568        2019.wsdm_conference-2019.40  \
1   1   7566  2008.sigirconf_conference-2008.116   
2   1  21754       2020.cikm_conference-2020.118   
3   1  33491   2018.wwwconf_conference-2018c.140   
4   1  14643       2019.wsdm_conference-2019.115   
5   1  21379       2009.cikm_conference-2009.191   
6   1  13109      2020.clef_conference-2020w.161   
7   1  13000       2020.clef_conference-2020w.52   
8   1  30099   2019.wwwconf_conference-2019c.232   
9   1   7607  2008.sigirconf_conference-2008.157   

                                                text   
0  Beyond News Contents: The Role of Social Conte...  \
1  Detecting synonyms in social tagging systems t...   
2  FANG: Leveraging Social Context for Fake News ...   
3  Fake News Detection in Social Networks via Cro...   
4  Fake News: Fundamental Theories, Detection Str...   
5  Context sensitive synonym discovery for web se...   
6  RMIT at PAN-CLEF 2020: Profiling Fake News Spr...   
7  Detecting Fake News Spreaders on Twitter Using...   
8  Fake News Detection: An Interdisciplinary Rese...   
9  Exploiting proximity feature in bigram languag...   

                                               title   
0  Beyond News Contents: The Role of Social Conte...  \
1  Detecting synonyms in social tagging systems t...   
2  FANG: Leveraging Social Context for Fake News ...   
3  Fake News Detection in Social Networks via Cro...   
4  Fake News: Fundamental Theories, Detection Str...   
5  Context sensitive synonym discovery for web se...   
6  RMIT at PAN-CLEF 2020: Profiling Fake News Spr...   
7  Detecting Fake News Spreaders on Twitter Using...   
8  Fake News Detection: An Interdisciplinary Rese...   
9  Exploiting proximity feature in bigram languag...   

                                            abstract  rank      score   
0                                                        0  29.858420  \
1  ABSTRACTCollaborative tagging used in online s...     1  28.648963   
2                                                        2  28.378120   
3  ABSTRACTOur work considers leveraging crowd si...     3  26.846009   
4  ABSTRACTThe explosive growth of fake news and ...     4  25.784173   
5  ABSTRACTWe propose a simple yet effective appr...     5  24.904962   
6  Automatic detection of fake news in social med...     6  24.251989   
7  In the present attempt, we have developed a fr...     7  23.769227   
8                                                        8  23.622076   
9  ABSTRACTLanguage modeling approaches have been...     9  23.529360   

                                               query  
0   relevant documents show the bigram of fake ne...  
1   relevant documents show the bigram of fake ne...  
2   relevant documents show the bigram of fake ne...  
3   relevant documents show the bigram of fake ne...  
4   relevant documents show the bigram of fake ne...  
5   relevant documents show the bigram of fake ne...  
6   relevant documents show the bigram of fake ne...  
7   relevant documents show the bigram of fake ne...  
8   relevant documents show the bigram of fake ne...  
9   relevant documents show the bigram of fake ne...

### Step 6: Persist Run

In [ ]:
print('Step 6: Persist Run.')

persist_and_normalize_run(run, output_file=output_directory, system_name='multi-field', depth=1000)

print('Done :)')